In [1]:
import os
import sys
from io import StringIO
import pandas as pd

In [11]:
hhresult_file = "/Users/michaeltisza/mike_tisza/sandbox/test_pyrgv1i/ct2_tmp/hhpred/AA_files/all_AA_seqs.no_phrogs.part_007.out.hhr"

input_dir = "/Users/michaeltisza/mike_tisza/sandbox/test_pyrgv1i/ct2_tmp/hhpred/AA_files/"

In [12]:
hhrout_list = []
for hhrout in os.listdir(input_dir):
    if hhrout.endswith('.hhr'):
        f = os.path.join(input_dir, hhrout)

        if os.path.isfile(f) and os.path.getsize(f) > 0:
            hhrout_list.append(f)

if not hhrout_list:
    print("no files found for pyhmmer in " + str(input_dir))
    exit

In [20]:
rows = []

for hhresult_file in hhrout_list:
    with open(hhresult_file, 'r') as hrh:
        for line in hrh:
            if line.startswith('Query         '):
                gene_name = line.strip('Query         ').split(' ')[0]
                #print(gene_name)
            if line.startswith('>'):
                full_desc = line.rstrip('\n').strip('>')
                accession = full_desc.split(' ; ')[0]
                annotation = full_desc.split(' ; ')[2]

                full_stats = next(hrh, '').strip()
                probability = full_stats.split('  ')[0].split('=')[1]
                evalue = full_stats.split('  ')[1].split('=')[1]
                Aligned_cols = full_stats.split('  ')[3].split('=')[1]
                row = [gene_name, accession, annotation, probability, evalue, Aligned_cols]

                rows.append(row)
                #print(full_stats)

hhpred_df = pd.DataFrame(rows, columns=['gene_name', 'evidence_accession', 'evidence_description', 
                                        'evidence_prob', 'evalue', 'Aligned_cols'])\
                                        .sort_values('evidence_prob').drop_duplicates('gene_name')

In [21]:
hhpred_df

,gene_name,evidence_accession,evidence_description,evidence_prob,evalue,Aligned_cols
30,test_pyrgv1i_3_8,PF16903.5,Major capsid protein N-terminus,100.00,8.8e-49,188
1,test_pyrgv1i_2_2,PF02443.15,Circovirus capsid protein,100.00,2e-46,193
18,test_pyrgv1i_1_2,PF02305.17,Capsid protein (F protein),100.00,3.4e-120,503
17,test_pyrgv1i_3_4,PF06244.12,Coiled-coil domain-containing protein 124 /Oxs1,82.50,0.046,46
39,test_pyrgv1i_4_11,PF10779.9,Haemolysin XhlA,83.25,0.031,35
38,test_pyrgv1i_1_6,PF06904.12,Extensin-like protein C-terminus,85.01,0.033,36
9,test_pyrgv1i_4_16,PF06023.12,CRISPR-associated exonuclease Csa1,89.55,0.0061,57
0,test_pyrgv1i_1_1,PF05840.13,Bacteriophage replication gene A protein (GPA),91.75,0.003,61
49,test_pyrgv1i_2_1,PF14532.6,Sigma-54 interaction domain,96.69,2.1e-06,83
28,test_pyrgv1i_3_5,PF00743.19,Flavin-binding monooxygenase-like,97.14,2.5e-07,43


In [ ]:
if not hhpred_df.empty:
    hhpred_output_file = os.path.join(out_dir, "hhpred_report_AAs.tsv")

    hhpred_df.to_csv(hhpred_output_file,
                            sep = "\t", index = False)